In [1]:
import tensorflow as tf
import numpy as np

In [ ]:
CKPT_FILE = '/home/commaai-03/Downloads/tmp/vgg_16.ckpt'
with tf.Session() as sess: 
    for var_name, _ in tf.contrib.framework.list_variables(CKPT_FILE): 
     var = tf.contrib.framework.load_variable(CKPT_FILE, var_name) 
     if 'biases' in var_name: 
         var_name = var_name.replace('biases', 'bias') 
     var = tf.Variable(var, name=var_name) 
    saver = tf.train.Saver() 
    sess.run(tf.global_variables_initializer()) 
    saver.save(sess, '/home/commaai-03/Downloads/tmp/vgg_16_1.ckpt') 

In [2]:
class VGG16():
        
    def repeat(self, inputs, times, filters, kernel_size, scope=None):
        
        with tf.variable_scope(scope):
            layers = []
            Conv2D = tf.keras.layers.Conv2D
            for time in range(times):
                layer = Conv2D(filters=filters, 
                               kernel_size=kernel_size, 
                               strides=(1,1), 
                               padding='same',
                               activation=tf.nn.relu, 
                               name='%s_%d' % (scope, time+1))
                layers.append(layer)
                net = layers[-1](inputs)
            return net
    
    def build(self, inputs, num_classes):
        
        with tf.variable_scope('vgg_16'):
            # Definition
            MaxPooling2D = tf.keras.layers.MaxPooling2D

            # First conv3-64 * 2
            net = self.repeat(inputs, 2, 64, [3, 3], scope='conv1')
            # MaxPooling 1: if strides is None, it will default to pool_size
            net = MaxPooling2D(pool_size=[2,2], 
                               strides=[2,2], 
                               padding='same')(net)
            # Second conv3-128 * 2
            net = self.repeat(net, 2, 128, [3,3], scope='conv2')
            net = MaxPooling2D(pool_size=[2,2], 
                               strides=[2,2], 
                               padding='same')(net)
            # Third conv3-256 * 3
            net = self.repeat(net, 3, 256, [3,3], scope='conv3')
            net = MaxPooling2D(pool_size=[2,2], 
                               strides=[2,2], 
                               padding='same')(net)
            # Fourth conv3-512 * 3
            net = self.repeat(net, 3, 512, [3,3], scope='conv4')
            net = MaxPooling2D(pool_size=[2,2], 
                               strides=[2,2], 
                               padding='same')(net)
            # Fifth conv3-512 * 3
            net = self.repeat(net, 3, 512, [3,3], scope='conv5')
            net = MaxPooling2D(pool_size=[2,2], 
                               strides=[2,2], 
                               padding='same')(net)

            # Conv2D instead of fully connected layers.
            # FC-4096
            net = tf.keras.layers.Conv2D(filters=4096, 
                                         kernel_size=[7,7], 
                                         strides=[1,1], 
                                         padding='valid', 
                                         activation=tf.nn.relu, 
                                         name='fc6')(net)
            net = tf.keras.layers.Dropout(0.5)(net)
            # FC-4096
            net = tf.keras.layers.Conv2D(filters=4096, 
                                         kernel_size=[1,1],
                                         strides=[1,1], 
                                         padding='valid', 
                                         activation=tf.nn.relu, 
                                         name='fc7')(net)
            net = tf.keras.layers.Dropout(0.5)(net)

            net = tf.keras.layers.Conv2D(filters=num_classes, 
                                         kernel_size=[1,1], 
                                         strides=[1,1], 
                                         padding='valid', 
                                         activation=None, 
                                         name='fc8')(net)
        return net

In [ ]:
CKPT_FILE = '/home/commaai-03/Downloads/tmp/vgg_16_1.ckpt'

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
net = VGG16().build(inputs=x, num_classes=1000)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(tf.global_variables())

W0929 14:32:09.764033 140618198177600 deprecation.py:506] From /home/commaai-03/.local/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[<tf.Variable 'vgg_16/conv1/conv1_1/kernel:0' shape=(3, 3, 3, 64) dtype=float32>, <tf.Variable 'vgg_16/conv1/conv1_1/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'vgg_16/conv1/conv1_2/kernel:0' shape=(3, 3, 3, 64) dtype=float32>, <tf.Variable 'vgg_16/conv1/conv1_2/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'vgg_16/conv2/conv2_1/kernel:0' shape=(3, 3, 64, 128) dtype=float32>, <tf.Variable 'vgg_16/conv2/conv2_1/bias:0' shape=(128,) dtype=float32>, <tf.Variable 'vgg_16/conv2/conv2_2/kernel:0' shape=(3, 3, 64, 128) dtype=float32>, <tf.Variable 'vgg_16/conv2/conv2_2/bias:0' shape=(128,) dtype=float32>, <tf.Variable 'vgg_16/conv3/conv3_1/kernel:0' shape=(3, 3, 128, 256) dtype=float32>, <tf.Variable 'vgg_16/conv3/conv3_1/bias:0' shape=(256,) dtype=float32>, <tf.Variable 'vgg_16/conv3/conv3_2/kernel:0' shape=(3, 3, 128, 256) dtype=float32>, <tf.Variable 'vgg_16/conv3/conv3_2/bias:0' shape=(256,) dtype=float32>, <tf.Variable 'vgg_16/conv3/conv3_3/kernel:0' shape=(3, 3, 128, 256) dtype

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
net = VGG16().build(inputs=x, num_classes=1000)

saver = tf.train.Saver()

with tf.Session() as sess:
    # Restore variables from ckpt file.
    tf.global_variables_initializer().run()
    
    saver.restore(sess, CKPT_FILE)
    print('[Tensorflow]: Model Restored.')
    print('FC8: %s' % net.eval())